In [114]:
# Have to install janitor package to run. Not included in anaconda.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import janitor

In [115]:
exec(open("../header.py").read())

# Import raw data

In [116]:
sample = True
header = Header(sample)

In [117]:
raw_df = pd.read_csv(header.clean_root("crsp_russ_ranks.csv"), parse_dates = ['date'])

C:\Users\Andrew\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [118]:
raw_df.head(2)

,permno,date,ticker_x,comnam,tsymbol,hexcd,cusip,prc,vol,ret,...,dollar_vol,ticker_y,russell1000,russell2000,name,r1000_wt,r2000_wt,russell_year,mkt_rank_desc,mkt_rank_asc
0,76272,1996-05-02,VPI,VINTAGE PETROLEUM INC,NaN,1,92746010,25.000,96000.0,0.020408,...,2.400000e+06,VPI,0.0,1.0,VINTAGE PETROLEUM IN,0.00,0.0556,1995,1183.0,1962.0
1,75510,1996-12-02,ADBE,ADOBE SYSTEMS INC,ADBE,3,00724F10,43.375,2332847.0,0.098101,...,1.011872e+08,ADBE,1.0,0.0,ADOBE SYSTEMS INC,0.04,0.0000,1996,375.0,450.0


In [119]:
raw_df.columns

Index(['permno', 'date', 'ticker_x', 'comnam', 'tsymbol', 'hexcd', 'cusip',
       'prc', 'vol', 'ret', 'bid', 'ask', 'shrout', 'numtrd', 'ewretd',
       'mktcap', 'year', 'month', 'yrmo', 'bid_ask_spread', 'turnover',
       'dollar_vol', 'ticker_y', 'russell1000', 'russell2000', 'name',
       'r1000_wt', 'r2000_wt', 'russell_year', 'mkt_rank_desc',
       'mkt_rank_asc'],
      dtype='object')

# Variables of interest

In [120]:
y_vars = ['bid_ask_spread', 'turnover', 'dollar_vol']
treatment_x_vars = ['russell2000']
running_x_vars = ['weight_rank', 'may_rank']

# Create running $x$ vars

In [121]:
df = raw_df.copy()

df['weight_rank_r1000'] = df\
    .groupby(['yrmo','year','russell1000'], as_index = False)['r1000_wt']\
    .transform(lambda x:x.rank(ascending = True))

df['weight_rank_r2000'] = df\
    .groupby(['yrmo','year','russell1000'], as_index = False)['r2000_wt']\
    .transform(lambda x:x.rank(ascending = False))

# Rely on fact that every stock is in either in r1000 or r2000 from data cleaning
df['weight_rank'] = np.where(df['russell1000'] == 1, df['weight_rank_r1000'], df['weight_rank_r2000'] * -1)

In [122]:
df['may_rank'] = np.where(df['russell1000'] == 1, df['mkt_rank_asc'], df['mkt_rank_desc'] * -1)

# Create $y$ vars

In [123]:
df['y1_spread'] = df['bid_ask_spread']/((df['bid']+df['ask'])/2)
df['y2_spread'] = df['bid_ask_spread']/df['prc']
df['y3_spread'] = df['bid_ask_spread']

In [124]:
# Fix bad formula
df['turnover'] = df['vol']/df['shrout']

df['y1_turnover'] = df\
    .groupby(['permno','russell_year'])['turnover']\
    .transform(lambda x:x.mean())
df['y2_turnover'] = df['turnover']

In [125]:
df['y1_dollar_vol'] = df\
    .groupby(['permno','russell_year'])['dollar_vol']\
    .transform(lambda x:x.mean())
df['y2_dollar_vol'] = df['dollar_vol']

# Difference-in-Mean Results

## Function

In [89]:
import scipy.stats

In [1]:
def diff_mean(df, y, running_x, treatment_x, bandwidth):
    # Restrict df[x] to bandwidth
    df_subset = df\
        .loc[lambda d:(d[running_x] <= bandwidth)&(d[running_x] >= -1*bandwidth)]
    
    # Show difference in features across treatment
    diff_means = df_subset\
        .groupby(treatment_x)\
        .agg(mean = (y, 'mean'),
             sd = (y, 'std'),
             count = (y, 'count'))\
        .reset_index()\
        .rename({'mean':y+'_mean',
                 'sd':y+'_sd',
                 'russell2000':'treatment'}, axis = 1)
    
    # Calculate statistical significance of difference
    mu_1 = diff_means.loc[lambda x:x.treatment == 1, y+'_mean'].values
    mu_2 = diff_means.loc[lambda x:x.treatment == 0, y+'_sd'].values
    sd_1 = diff_means.loc[lambda x:x.treatment == 1, y+'_sd'].values
    sd_2 = diff_means.loc[lambda x:x.treatment == 0, y+'_sd'].values
    n_1 = diff_means.loc[lambda x:x.treatment == 1, 'count'].values
    n_2 = diff_means.loc[lambda x:x.treatment == 0, 'count'].values
    
    t_stat = (mu_1 - mu_2)/np.sqrt((sd_1**2/n_1) + (sd_2**2/n_2))
    p_val = scipy.stats.norm.sf(np.abs(t_stat))
    
    return diff_means, (t_stat, p_val)

## Spread

In [2]:
r1 = diff_mean(df, y = 'y1_spread', running_x = 'weight_rank', treatment_x = 'russell2000', bandwidth = 50)
r1[0]

NameError: name 'df' is not defined

In [92]:
diff_mean(df, y = 'y2_spread', running_x = 'weight_rank', treatment_x = 'russell2000', bandwidth = 50)

(   treatment  y2_spread_mean  y2_spread_sd  count
 0        0.0        0.002233      0.018116   6577
 1        1.0        0.000259      0.000639   6568,
 (array([-56.52602274]), array([0.])))

In [93]:
# Economic significance
diff_mean(df, y = 'y3_spread', running_x = 'weight_rank', treatment_x = 'russell2000', bandwidth = 50)

(   treatment  y3_spread_mean  y3_spread_sd  count
 0        0.0        0.011456      0.016062   6577
 1        1.0        0.006379      0.008627   6568,
 (array([-34.57306658]), array([3.2090647e-262])))

## Turnover

In [94]:
diff_mean(df, y = 'y1_turnover', running_x = 'weight_rank', treatment_x = 'russell2000', bandwidth = 50)

(   treatment  y1_turnover_mean  y1_turnover_sd  count
 0        0.0          8.875354       14.353187   6577
 1        1.0         11.969459       11.806245   6568,
 (array([-9.52373566]), array([8.35315453e-22])))

In [95]:
diff_mean(df, y = 'y2_turnover', running_x = 'weight_rank', treatment_x = 'russell2000', bandwidth = 50)

(   treatment  y2_turnover_mean  y2_turnover_sd  count
 0        0.0          8.847543       22.343737   6577
 1        1.0         12.692259       21.077053   6568,
 (array([-24.77062418]), array([9.2946729e-136])))

In [99]:
diff_mean(df, y = 'y2_turnover', running_x = 'weight_rank', treatment_x = 'russell2000', bandwidth = 50)

(   treatment  y2_turnover_mean  y2_turnover_sd  count
 0        0.0          8.847543       22.343737   6577
 1        1.0         12.692259       21.077053   6568,
 (array([-24.77062418]), array([9.2946729e-136])))

## Dollar volume

In [100]:
diff_mean(df, y = 'y1_dollar_vol', running_x = 'weight_rank', treatment_x = 'russell2000', bandwidth = 50)


(   treatment  y1_dollar_vol_mean  y1_dollar_vol_sd  count
 0        0.0        1.102144e+07      1.867413e+07   6577
 1        1.0        2.239079e+07      3.547478e+07   6568,
 (array([11.41329339]), array([1.79439481e-30])))

In [101]:
diff_mean(df, y = 'y2_dollar_vol', running_x = 'weight_rank', treatment_x = 'russell2000', bandwidth = 50)


(   treatment  y2_dollar_vol_mean  y2_dollar_vol_sd  count
 0        0.0        9.291123e+06      2.170489e+07   6577
 1        1.0        2.675695e+07      6.729659e+07   6568,
 (array([13.34780822]), array([6.09927456e-41])))

# RDD Results

## Function

In [126]:
import statsmodels.formula.api as smf

In [136]:
def rdd(df, y, running_x, treatment_x, bandwidth, polynomials = 3):
    # Restrict df[x] to bandwidth
    df_subset = df.copy()\
        .loc[lambda d:(d[running_x] <= bandwidth)&(d[running_x] >= -1*bandwidth)]
    
    # Add on the polynomial terms on both sides of the treatment
    ind_vars = [running_x, treatment_x, 'interact']
    df_subset['interact'] = df_subset[running_x] * df_subset[treatment_x]
    
    for p in range(2,polynomials+1):
        df_subset[f'{running_x}_p{p}'] = df_subset[running_x]**p
        df_subset[f'interact_p{p}'] = df_subset[f'{running_x}_p{p}']*df_subset[treatment_x]
        ind_vars.append(f'{running_x}_p{p}')
        ind_vars.append(f'interact_p{p}')
        
    ind_vars = '+'.join(ind_vars)
    print(ind_vars)
    
    rdd = smf.ols(f'{y} ~ {ind_vars}', data = df_subset).fit()
    
    return rdd

In [137]:
r1 = rdd(df, y = 'y1_dollar_vol', running_x = 'weight_rank', treatment_x = 'russell2000', bandwidth = 50, polynomials = 3)

weight_rank+russell2000+interact+weight_rank_p2+interact_p2+weight_rank_p3+interact_p3


In [145]:
pd.DataFrame({'coef':r1.params, 't_stat':r1.tvalues}).reset_index()

,index,coef,t_stat
0,Intercept,7.562168e+06,5.128451
1,weight_rank,2.577434e+05,1.037734
2,russell2000,5.099865e+07,24.489538
3,interact,3.805941e+06,10.840113
4,weight_rank_p2,-8.309971e+03,-0.737573
5,interact_p2,1.348849e+05,8.459721
6,weight_rank_p3,1.240248e+02,0.852481
7,interact_p3,1.144499e+03,5.552413


# RDD Variation
Include controls for size. Though that should have the opposite effect.

In [126]:
import statsmodels.formula.api as smf

In [148]:
def rdd(df, y, running_x, treatment_x, bandwidth, polynomials = 3):
    # Restrict df[x] to bandwidth
    df_subset = df.copy()\
        .loc[lambda d:(d[running_x] <= bandwidth)&(d[running_x] >= -1*bandwidth)]
    
    # Add on the polynomial terms on both sides of the treatment
    ind_vars = [running_x, treatment_x, 'interact', 'mktcap']
    df_subset['interact'] = df_subset[running_x] * df_subset[treatment_x]
    
    for p in range(2,polynomials+1):
        df_subset[f'{running_x}_p{p}'] = df_subset[running_x]**p
        df_subset[f'interact_p{p}'] = df_subset[f'{running_x}_p{p}']*df_subset[treatment_x]
        ind_vars.append(f'{running_x}_p{p}')
        ind_vars.append(f'interact_p{p}')
        
    ind_vars = '+'.join(ind_vars)
    print(ind_vars)
    
    rdd = smf.ols(f'{y} ~ {ind_vars}', data = df_subset).fit()
    
    return pd.DataFrame({'coef':rdd.params, 't_stat':rdd.tvalues}).reset_index()

In [149]:
rdd(df, y = 'y1_dollar_vol', running_x = 'weight_rank', treatment_x = 'russell2000', bandwidth = 50, polynomials = 3)

weight_rank+russell2000+interact+mktcap+weight_rank_p2+interact_p2+weight_rank_p3+interact_p3


,index,coef,t_stat
0,Intercept,-1.326124e+06,-0.959110
1,weight_rank,-1.618802e+05,-0.701457
2,russell2000,3.176761e+07,16.057625
3,interact,3.059719e+06,9.374841
4,mktcap,9.880367e+00,45.874249
5,weight_rank_p2,-4.105878e+02,-0.039247
6,interact_p2,9.045180e+04,6.097235
7,weight_rank_p3,7.558128e+01,0.559536
8,interact_p3,8.104783e+02,4.231986


In [150]:
rdd(df, y = 'y1_dollar_vol', running_x = 'may_rank', treatment_x = 'russell2000', bandwidth = 50, polynomials = 3)

may_rank+russell2000+interact+mktcap+may_rank_p2+interact_p2+may_rank_p3+interact_p3


,index,coef,t_stat
0,Intercept,-3.099740e+06,-0.608466
1,may_rank,5.658745e+05,0.656424
2,russell2000,8.612586e+08,17.014550
3,interact,9.251901e+07,16.406562
4,mktcap,1.382080e+01,7.329906
5,may_rank_p2,-4.394339e+04,-1.073113
6,interact_p2,3.144189e+06,15.749289
7,may_rank_p3,6.577932e+02,1.182006
8,interact_p3,3.154517e+04,14.228222


,index,coef,t_stat
0,Intercept,7.562168e+06,5.128451
1,weight_rank,2.577434e+05,1.037734
2,russell2000,5.099865e+07,24.489538
3,interact,3.805941e+06,10.840113
4,weight_rank_p2,-8.309971e+03,-0.737573
5,interact_p2,1.348849e+05,8.459721
6,weight_rank_p3,1.240248e+02,0.852481
7,interact_p3,1.144499e+03,5.552413


# Save results